In [1]:
import os
os.chdir("..")

In [3]:
import pandas as pd 
path = "data\MF Customer Details.xlsx"

cust_data =  pd.read_excel(path)

cust_data.head()



,Customer ID,Occupation,Risk Appetite
0,131015,Salaried,High
1,131006,Salaried,High
2,131005,Salaried,Low
3,130988,Salaried,Medium
4,130986,Salaried,Low


In [17]:
red_data = pd.read_csv("data\MF_Model_base.csv",
                 parse_dates= {"date" : ["Year","Month"]})
red_data.info()

C:\Users\arunk\AppData\Local\Temp\ipykernel_2156\1581842844.py:1: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  red_data = pd.read_csv("data\MF_Model_base.csv",


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 946207 entries, 0 to 946206
Data columns (total 12 columns):
 #   Column           Non-Null Count   Dtype         
---  ------           --------------   -----         
 0   date             946207 non-null  datetime64[ns]
 1   Customer ID      946207 non-null  int64         
 2   FA ID            946207 non-null  int64         
 3   Scheme Code      946207 non-null  int64         
 4   Txn. No          946207 non-null  object        
 5   Txn. Type        946207 non-null  object        
 6   Txn. Sub Type    946207 non-null  object        
 7   Units            946207 non-null  float64       
 8   Purchase Amount  946207 non-null  float64       
 9   City             943217 non-null  object        
 10  Pin Code         945645 non-null  object        
 11  Date of Birth    937774 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(3), object(6)
memory usage: 86.6+ MB


In [19]:
min_date = red_data["date"].min()
max_date = red_data["date"].max()
print(min_date,max_date)

2013-12-01 00:00:00 2022-12-01 00:00:00
